# Red Wine Quality

Kaggle link: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

## Preliminaries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import metrics
import random
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import torch
from torch import nn
from sklearn.model_selection import train_test_split
#import wandb
#wandb.init(project="simple_regression")

import os
for dirname, _, filenames in os.walk('./'): # '/kaggle/input'
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./simple-regression.ipynb
./winequality-red.csv
./wandb/debug-internal.log
./wandb/debug-cli.lorenzozanolin.log
./wandb/debug.log
./wandb/run-20231013_152213-1mdmbnvn/run-1mdmbnvn.wandb
./wandb/run-20231013_152213-1mdmbnvn/logs/debug-internal.log
./wandb/run-20231013_152213-1mdmbnvn/logs/debug.log
./wandb/run-20231013_152213-1mdmbnvn/files/requirements.txt
./wandb/run-20231013_152213-1mdmbnvn/files/output.log
./wandb/run-20231013_152213-1mdmbnvn/files/config.yaml
./wandb/run-20231013_152213-1mdmbnvn/files/wandb-metadata.json
./wandb/run-20231013_153847-h2jvr2b3/run-h2jvr2b3.wandb
./wandb/run-20231013_153847-h2jvr2b3/logs/debug-internal.log
./wandb/run-20231013_153847-h2jvr2b3/logs/debug.log
./wandb/run-20231013_153847-h2jvr2b3/files/requirements.txt
./wandb/run-20231013_153847-h2jvr2b3/files/output.log
./wandb/run-20231013_153847-h2jvr2b3/files/config.yaml
./wandb/run-20231013_153847-h2jvr2b3/files/wandb-summary.json
./wandb/run-20231013_153847-h2jvr2b3/files/wandb-metadata.json
./wand

## Data Processing

In [3]:
train_data = pd.read_csv('./winequality-red.csv')    #'/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv'
train_data.head()
train_data.shape

(1599, 12)

First we will split the training set in two parts: *training* and *test* set.

In [4]:

X_train,X_test,y_train,y_test = train_test_split(train_data.iloc[:,:-1],train_data.iloc[:,-1],test_size=0.25) #splitting into training set and test set

#convert the sets from ndarray to tensor
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)


Now we will *normalize* features in the following manner:
- mean and std will be computed on the *training* set
- these values will be used to compute the normalization on the *test* set

In [5]:
def normalize_features(X_train, X_test, y_train, y_test):
    global train_mean, train_std, pred_mean, pred_std

    train_mean = [0,0,0,0,0,0,0,0,0,0,0] 
    train_std =[0,0,0,0,0,0,0,0,0,0,0]
    
    
    for i in range(0, X_train.shape[1]-1):
        train_mean[i] = X_train[i].mean()
        train_std[i] = X_train[i].std()
    
        X_train = (X_train - train_mean[i])/train_std[i]
        X_test = (X_test - train_mean[i])/train_std[i]
    
    print(X_train)
    
    pred_mean = y_train.mean()
    pred_std = y_train.std()
    
    y_train = (y_train - pred_mean) / pred_std
    y_train = torch.tensor(y_train,dtype=torch.float32)

    
    y_test = (y_test - pred_mean) / pred_std
    y_test = torch.tensor(y_test,dtype=torch.float32)
    return X_train, X_test, y_train, y_test

def denormalize_features(X_train, X_test, y_train, y_test):
    global train_mean, train_std, pred_mean, pred_std
    
    
    for i in range(0, X_train.shape[1]-1):
        X_train = (X_train*train_std[i])+train_mean[i]
        X_test = (X_test*train_std[i])+train_mean[i]
    
    y_train = (y_train * pred_std)+pred_mean
    y_train = torch.tensor(y_train,dtype=torch.float32)

    
    y_test = (y_test*pred_std) + pred_mean
    y_test = torch.tensor(y_test, dtype=torch.float32)
    return X_train, X_test, y_train, y_test

X_train_N,X_test_N,y_train_n,y_test_n = normalize_features(X_train,X_test,y_train,y_test)   #normalizing the dataset

tensor([[ 2.4856e-01, -5.2750e-01, -5.0018e-01,  ..., -3.4746e-01,
         -4.9832e-01,  1.2439e-01],
        [ 7.4719e-02, -5.0950e-01, -5.2688e-01,  ..., -3.4870e-01,
         -5.0701e-01,  8.7136e-02],
        [-4.9450e-02, -5.1943e-01, -5.2378e-01,  ..., -3.3876e-01,
         -5.1757e-01,  3.7468e-02],
        ...,
        [-1.3016e-01, -5.1012e-01, -5.4116e-01,  ..., -3.4249e-01,
         -5.1260e-01,  2.5051e-02],
        [ 2.1758e-04, -5.0825e-01, -5.3433e-01,  ..., -3.4621e-01,
         -5.1508e-01,  7.4719e-02],
        [-1.2395e-01, -4.9739e-01, -5.4613e-01,  ..., -3.2759e-01,
         -5.1198e-01,  1.1818e-01]])


/var/folders/47/wgqjbdx10d91c2t28x5n0s5m0000gn/T/ipykernel_64120/3525117486.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train,dtype=torch.float32)
/var/folders/47/wgqjbdx10d91c2t28x5n0s5m0000gn/T/ipykernel_64120/3525117486.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test,dtype=torch.float32)


## Training

Initialize the weights and bias of the linear regression

In [6]:
n_features = X_train.shape[1] 
weights = torch.randn((n_features),requires_grad=True) # initialize a random tensor of weights, one weight for each feature
bias = torch.zeros(1, requires_grad=True)
#weights,bias

Some functions definitions

In [7]:
from matplotlib import pyplot as plt


def linreg(X, w, b):        #linear regression
    return torch.matmul(X, w) + b

def sgd(params, lr, batch_size):    #we will use stochastic GD
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()
            
criterion = nn.MSELoss()    #the loss function will be mean squared error

def data_iter(batch_size, features, labels):    #to divide a single batch in multiple minibatches
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i +
                                                   batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]
        
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

def print_result(y_true, y_pred):
    plt.clf()
    plt.plot(y_pred, 'ro', label='Predictions', alpha=0.5)
    plt.plot(y_true, 'go', label='True', alpha=0.5)
    plt.legend(loc='best')
    plt.colorbar
    plt.show()
    
def print_correctness(ground_truth,predicted_labels):
    print(metrics.accuracy_score(ground_truth,predicted_labels))

Training loop

In [8]:
# MINI BATCH, calculate for each round the derivative for each minibatch, and then sum up them together
num_iterations = 512
batch_size = 16
lr = 1e-3
print('Training loss:')
for i in range(num_iterations):
    for X, y in data_iter(batch_size, X_train_N, y_train_n):    
        loss = squared_loss(linreg(X,weights,bias),y)    #calculate the prediction, i.e. X(train features) * weights +b; then the loss w.r.t. labels
        loss.sum().backward() #derivate calc
        sgd([weights,bias],lr,batch_size)   #update weights
    with torch.no_grad():
        train_l = squared_loss(linreg(X, weights, bias), y) #loss of the final batch of the round
        print(f'epoch {i + 1}, loss {float(train_l.mean()):f}')
    

Training loss:
epoch 1, loss 2.906533
epoch 2, loss 1.771536
epoch 3, loss 0.613548
epoch 4, loss 1.271393
epoch 5, loss 0.445122
epoch 6, loss 0.497496
epoch 7, loss 0.604710
epoch 8, loss 0.459592
epoch 9, loss 0.415414
epoch 10, loss 0.817154
epoch 11, loss 1.054489
epoch 12, loss 1.004345
epoch 13, loss 0.857615
epoch 14, loss 0.454113
epoch 15, loss 0.371683
epoch 16, loss 0.603150
epoch 17, loss 0.552494
epoch 18, loss 0.716668
epoch 19, loss 0.558111
epoch 20, loss 0.471840
epoch 21, loss 0.714187
epoch 22, loss 0.495846
epoch 23, loss 0.667086
epoch 24, loss 0.411321
epoch 25, loss 0.573872
epoch 26, loss 0.605541
epoch 27, loss 0.291797
epoch 28, loss 0.519602
epoch 29, loss 0.627199
epoch 30, loss 0.449147
epoch 31, loss 0.331730
epoch 32, loss 0.677108
epoch 33, loss 0.648507
epoch 34, loss 0.526581
epoch 35, loss 0.530271
epoch 36, loss 0.264221
epoch 37, loss 0.516984
epoch 38, loss 0.188744
epoch 39, loss 0.358312
epoch 40, loss 0.553259
epoch 41, loss 0.448012
epoch 42, 

## Testing

We need to calculate the loss also over the test set

In [9]:
# test on the test dataset
with torch.no_grad():
    test_l = 0
    for X, y in data_iter(batch_size, X_test_N, y_test_n):
        test_l += squared_loss(linreg(X, weights, bias), y).sum()
    test_l /= len(X_test)
    print(f'loss on the test dataset {float(test_l):f}')

loss on the test dataset 0.505891


## Results

Finally, the obtained predictions are the following

In [12]:
test_predictions_N = []
with torch.no_grad():
    for X in X_test_N:
        test_predictions_N.append(linreg(X,weights,bias))

_,_,test_predictions,y_test = denormalize_features(X_train_N,X_test_N,torch.tensor(test_predictions_N),y_test_n)

test_predictions = test_predictions.round()

with torch.no_grad():
    print_correctness(y_test,test_predictions)  #prints accuracy

0.455


/var/folders/47/wgqjbdx10d91c2t28x5n0s5m0000gn/T/ipykernel_64120/3525117486.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train,dtype=torch.float32)
/var/folders/47/wgqjbdx10d91c2t28x5n0s5m0000gn/T/ipykernel_64120/3525117486.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test, dtype=torch.float32)
